In [ ]:
from pdstools import IH

import plotly.io as pio
import plotly as plotly

plotly.offline.init_notebook_mode()
pio.renderers.default = "vscode"

# IH Example Analysis

This notebook uses sample data shipped with PDStools. Replace with actual IH data.

In [2]:
ih = IH.from_ds_export(
    "../../data/Data-pxStrategyResult_pxInteractionHistory_20210101T010000_GMT.zip"
)

At first, take a look into the IH dataframe, explore the columns, outcome types and business structure

In [ ]:
ih.data.head().collect()

In [ ]:
# df.groupby(['pyIssue', 'pyGroup', 'pyDirection', 'pyChannel', 'pyName', 'pyOutcome']).count()[[
#     'pxInteractionID']].rename(columns={'pxInteractionID':'Count'})

# TODO tree map
import plotly.express as px

plot_data = ih.aggregates.summary_outcomes(
    by=["Issue", "Group", "Direction", "Channel", "Name"]
).collect()
fig = px.treemap(
    plot_data,
    path=[px.Constant("ALL")]
    + ["Outcome"]
    + ["Issue", "Group", "Direction", "Channel", "Name"],
    values="Count",
    color="Count",
    branchvalues="total",
    # color_continuous_scale=px.colors.sequential.RdBu_r,
    # title=title,
    # hover_data=[
    #     f"StdErr_{metric}",
    #     f"Positives_{metric}",
    #     f"Negatives_{metric}",
    # ],
    height=640,
    template="pega",
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textinfo="label+value+percent parent")
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig

Use "plot_daily_accept_rate" to plot accept rate per day to understand how accept rates changed over time. To define accept rate, enter the positive (here: Accepted) and negative (here: Rejected) behaviour in the function. use kwargs to customize the graph. If the time ticks on the x axis are too many, shrink them using 'shrinkTicks'. If data is missing in certain days, force the graph make gaps for the missing days by setting 'allTime':True. you can also define hue

In [ ]:
# plot_daily_accept_rate(
#     df,
#     "Accepted",
#     "Rejected",
#     **{"hue": ["pyChannel"], "allTime": True, "shrinkTicks": True},
# )
# TODO more or less fits the engagement trend charts
ih.plots.success_rates_trend_line(by="Channel")

The above graph provides detailed metric behavior over time. Instead of Accept, you can use other outcome types. To get a rolled up view, plot the accept rate graph based on a weekly axis. The week values are calculated based on the starting date of the IH file

In [ ]:
# plot_weekly_accept_rate(
#     df, "Accepted", "Rejected", **{"showOutlier": True, "hue": "pyDirection"}
# )
# Same

The above graphs provide insight into the accept rates on daily or weekly basis. ADM models however, take all time data every update cycle, to generate bubble charts. To view the historical cumulative accept rate, use the function below. If choosing a single model, this graph will be as if you had ADM success rate captured over time. Set 'showOutlier' to True to view outlier values

In [ ]:
plot_daily_cumulative_accept_rate(
    df[df["pyName"] == "UPlusPersonal"],
    "Accepted",
    "Rejected",
    **{
        "allTime": True,
        "shrinkTicks": True,
        "showOutlier": True,
        "title": "Proposition: UPlusPersonal",
    },
)
# Not sure how useful that really is

The above graph can be done in various granularity level. For example the below graph shows the cumulative accept rate over time across all the offers

In [ ]:
plot_daily_cumulative_accept_rate(
    df,
    "Accepted",
    "Rejected",
    **{"allTime": True, "shrinkTicks": True, "showOutlier": True},
)

Below graph shows the cumulative accept rate per pyGroup, pyDirection and pyChannel

In [ ]:
plot_daily_cumulative_accept_rate(
    df,
    "Clicked",
    "NoResponse",
    **{
        "hue": ["pyGroup", "pyDirection", "pyChannel"],
        "allTime": True,
        "shrinkTicks": True,
    },
)

In addition to accept rate, it is important to track other outcome values over time. Use 'daily' or 'weekly' to set the granularity of time axis. Instead of 'Accepted', other outcome labels can be explored over time

In [ ]:
plot_outcome_count_time(
    df, "Accepted", "weekly", **{"hue": "pyIssue", "allTime": True, "shrinkTicks": True}
)

# A count trend seems useful to, not just the CTR

below graphs puts a couple of graphs together to provide better insight at the offer level to be able to compare the accept rate, accept count and total responses per model

In [ ]:
plot_df = get_accept_rate(
    df[df["pyDirection"] == "Inbound"], "Accepted", "Rejected", "pyName"
)

fig, ax = plt.subplots(2, 1, figsize=(13, 9), sharex=True, gridspec_kw={"hspace": 0.05})
sort = plot_df.sort_values("Accept Rate (%)", ascending=False)["pyName"].tolist()
sns.barplot(x="pyName", y="Accept Rate (%)", data=plot_df, ax=ax[0], order=sort)
sns.barplot(x="pyName", y="Accepted", data=plot_df, ax=ax[1], order=sort)
sns.pointplot(x="pyName", y="Total", data=plot_df, ax=ax[1], order=sort)
for x in ax[1].get_xmajorticklabels():
    x.set_rotation(90)
ax[0].set_xlabel("")
ax[1].text(2, 2000, "The bars show the accepts\nThe line shows accept+reject")
ax[0].set_ylabel("Accept Rate (%)", fontsize=13)
ax[1].set_ylabel("Accepts", fontsize=13)
ax[0].set_title("Offers within Inbound direction")

# Not sure about this

Another insightful graph is to see what share of a given outcome label, each offer(or direction or channel) has. For example the below graph shows that of all the historical 'Accepted' labels, 'UPlusGold' proposition has a little over 50% of all the 'Accepted' outcomes. 'UPlusFinPersonal' has roughly 10% of all time Accepted outcomes. instead of proposition level, you can set other levels (channel, direction etc).

In [ ]:
plot_outcome_share_graph(df[df["pyChannel"] == "Web"], "Accepted", "pyName", "pyGroup")

# Wouldn't the tree show this?

It is also possible to see how the outcome share of a given proposition (or channel etc.) changed over time

In [ ]:
click_share_name_daily = get_outcome_share_time(
    df[df["pyChannel"] == "Web"], "Clicked", "pyName", time="daily"
)
click_share_name_weekly = get_outcome_share_time(
    df[df["pyChannel"] == "Web"], "Clicked", "pyName", time="weekly"
)

The graph below shows among offer within Web channel, what share of Clicked outcome labels belonged to UPlusGold proposition every day. It can be seen that the value dropped significantly on 12-23

In [ ]:
get_daily_graph(
    click_share_name_daily[click_share_name_daily["pyName"] == "UPlusGold"],
    "Date",
    "Clicked Share (%)",
    **{"shrinkTicks": True},
)

same graph can be viewed on a weekly basis

In [ ]:
get_daily_graph(
    click_share_name_weekly[click_share_name_weekly["pyName"] == "UPlusGold"],
    "Week",
    "Clicked Share (%)",
    **{"shrinkTicks": True},
)

The graph below shows the Accepted share between two directions: Inbound/Outbound. Of course in this case because there are only 2 directions, when one graph goes up, the other has to go down so the sum of the two per day would be 100%

In [ ]:
click_share_direction_daily = get_outcome_share_time(
    df, "Accepted", "pyDirection", time="daily"
)

In [ ]:
get_daily_graph(
    click_share_direction_daily,
    "Date",
    "Accepted Share (%)",
    **{"shrinkTicks": True, "hue": "pyDirection"},
)

The above graph can help identify how things evolve as a whole. It helps identify when the share for one direction (or channel etc.) goes down, which channel takes over goes down, which other 

same graph can be done for pyName as well. However, since there are usually so many pyNames, it would be hard to follow up and identify which offer had the highest share over time, and when an offer's share drops, which other offer takes over. So instead of looking over time, the below graph calculates a delta between the share percentage across two time frames. This is significanlty helpful when things in the strategy changes (priotitization, eligibility etc.) it helps identify how the system reacts once there is a change introduced.

In [ ]:
plot_share_delta_graph(
    df[df["pyChannel"] == "SMS"].reset_index(drop=True), "Clicked", "pyName", dates=4
)

# A delta view of actions vs a condition (eg experiment) could be neat, showing a delta of offer counts
# maybe a variant of the regular bar charts

In the above graph, the clicked outcome shares for MasterCardGold has increased by 5% recently. The time range can be specified either by defining a lookback window (in that case only enter an integer) or by a list of two tuples where the first tuple represents the earlier time range and the second tuple represent the recent time range

# Performance

Model performance over time
